In [57]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from pytube import YouTube
from pymongo import MongoClient
from tqdm import tqdm
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import urllib.request
import datetime
import copy

In [58]:
# MongoDB 클라이언트 및 컬렉션 설정
client = MongoClient(host="1.234.51.110", port=38019, username='clawling', password='goodtime**95')
db = client['YouTube_crawling']
collection = db['OneAsia_YouTube_shorts_crawling']
data_list = list(collection.find({"검색키워드" : "釜山ワンアジアフェスティバル"}, {"_id": 1, "URL": 1}))

In [59]:
data_list1 = copy.deepcopy(data_list)

In [60]:
driver = webdriver.Chrome()

In [61]:
# 좋아요 개수를 변환 (ex) 1.3천 → 1300, 1.3만 → 13000 형식으로 변환)
def convert_like_count(like_count_text):
    if '천' in like_count_text:
        count = int(float(like_count_text.replace('천', '')) * 1000)
    elif '만' in like_count_text:
        count = int(float(like_count_text.replace('만', '')) * 10000)
    else:
        count = int(like_count_text)
    return count

In [62]:
def process_link(url):
    try:
        driver.get(url)

        time.sleep(3)

        # 좋아요 개수 가져오기
        try:
            like_count_span = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div[aria-hidden='true'] span.yt-core-attributed-string"))
            )
    
            # 좋아요 개수를 추출합니다.
            like_count_text = like_count_span.text
            like_count = convert_like_count(like_count_text)

        except Exception as e:
            # 좋아요 개수를 찾을 수 없으면 0을 할당
            like_count = 0

        print(f"Like count: {like_count}")

        # Mongodb에 적재
        collection.update_one({"URL": url}, {"$set": {
            "좋아요": like_count
        }})

        time.sleep(3)

    except Exception as e:
        print("URL 처리 중 오류 발생:", e)

In [63]:
for data in tqdm(data_list1, desc="Processing Links", unit="link"):
    url = data['URL']
    process_link(url)

client.close()
driver.quit()

Processing Links:   0%|          | 0/10 [00:00<?, ?link/s]

Like count: 3


Processing Links:  10%|█         | 1/10 [00:08<01:14,  8.31s/link]

Like count: 7


Processing Links:  20%|██        | 2/10 [00:15<01:01,  7.74s/link]

...

Processing Links:  80%|████████  | 8/10 [01:00<00:15,  7.53s/link]

Like count: 70


Processing Links:  90%|█████████ | 9/10 [01:08<00:07,  7.49s/link]

Like count: 530


Processing Links: 100%|██████████| 10/10 [01:15<00:00,  7.54s/link]
